In [ ]:
import g2o
import scipy
import numpy as np
import matplotlib.pyplot as plt

from icp import icp
from graph import Graph
from dataset_intel import Dataset_intel

In [ ]:
# Variables
max_iterations = 2500
thres = 0.15
lc_num = 0

max_x = -float('inf')
max_y = -float('inf')
min_x = float('inf')
min_y = float('inf')

# Load dataset
odoms_laser, lasers = Dataset_intel('data').get_dataset()

# Create graph
graph = Graph()

# Initial state
pose = np.eye(3)
id = 0
graph.add_vertex(id, g2o.SE2(g2o.Isometry2d(pose)), fixed=True)

init_pose = np.eye(3)
vertex_idx = 1
registered_lasers = []
registered_idxs = []
registered_lasers.append(lasers[0])
registered_idxs.append(0)
vertex_id_odom_idx = []

# add odom to graph
for odom_idx, odom in enumerate(odoms_laser):
    if odom_idx==max_iterations:
        break
    if odom_idx==0:
        prev_odom = odom[odom_idx].copy()
        prev_idx = 0
        continue

    # Check if the pose moved
    do = odom - prev_odom
    if np.linalg.norm(do[:2])>0.4 or abs(do[2])>0.2:

        # (2, 180)
        A = lasers[prev_idx]
        B = lasers[odom_idx]
        registered_lasers.append(B)
        registered_idxs.append(odom_idx)
        dx, dy, dtheta = do[0], do[1], do[2]
        init_pose = np.array([[np.cos(dtheta), -np.sin(dtheta), dx], [np.sin(dtheta), np.cos(dtheta), dy],[0, 0, 1]])
        with np.errstate(all='raise'):
            try:
                T, distances, iterations,information = icp(B.T, A.T, init_pose, max_iterations=80, tolerance=0.0001)

            except Exception as e:
                print(odom_idx, e, A.shape, B.shape)
                assert 1==0
                continue
        
        # Update the pose
        pose = np.matmul(pose, T)
        graph.add_vertex(vertex_idx, g2o.SE2(g2o.Isometry2d(pose)))
        vertex_id_odom_idx.append(odom_idx)

        rk = g2o.RobustKernelDCS()

        graph.add_edge([vertex_idx-1, vertex_idx],
                                 g2o.SE2(g2o.Isometry2d(T)),
                                 information, robust_kernel=rk)
        
        # Update
        prev_odom = odom
        prev_idx = odom_idx

        # Loop closure
        if vertex_idx > 10 and not vertex_idx % 10:
            poses = [graph.get_pose(idx).to_vector()[:2] for idx in range(vertex_idx-1)]

            kd = scipy.spatial.cKDTree(poses)
            x, y, theta = graph.get_pose(vertex_idx).to_vector()
            direction = np.array([np.cos(theta), np.sin(theta)])
            idxs = kd.query_ball_point(np.array([x,y]), r=4.25, p=2.)
            for idx in idxs:
                A = registered_lasers[idx]
                with np.errstate(all='raise'):
                    try:
                        T, distances, iterations, information = icp(A.T, B.T, np.eye(3), max_iterations=80, tolerance=0.0001)

                    except Exception as e:
                        print(odom_idx, e, A.shape, B.shape)
                        continue

                if np.mean(distances) < thres:
                    dist = np.linalg.norm(T[:2,2])
                    print(f'odom_idx: {odom_idx}, vertex_idx: {vertex_idx}, lc_num: {lc_num}, dist: {dist}, Edge added.')
                    lc_num+=1

                    rk = g2o.RobustKernelDCS()
                    graph.add_edge([vertex_idx, idx], g2o.SE2(g2o.Isometry2d(T)), information, robust_kernel=rk)

            graph.optimize()
            pose = graph.get_pose(vertex_idx).to_isometry().matrix()

        vertex_idx+=1

In [ ]:
traj = []
point_cloud = []

for idx in range(vertex_idx):
    fig, ax = plt.subplots( nrows=1, ncols=1 )
    
    traj.append(graph.get_pose(idx).to_vector()[:2])

    x = graph.get_pose(idx)
    r = x.to_isometry().R
    t = x.to_isometry().t
    filtered = registered_lasers[idx].T
    filtered = filtered[np.linalg.norm(filtered, axis=1) < 80]
    point_cloud.append((r @ filtered.T + t[:, np.newaxis]).T)

    # Trajectory
    traj_np = np.array(traj)
    ax.plot(traj_np[:, 0], traj_np[:, 1], '-g')
    
    # Point cloud
    point_cloud_np = np.vstack(point_cloud)
    xyreso = 0.01 # Map resolution (m)
    point_cloud_np = (point_cloud_np / xyreso).astype('int')
    point_cloud_np = np.unique(point_cloud_np, axis=0)
    point_cloud_np = point_cloud_np * xyreso
    
    # Plot
    ax.plot(point_cloud_np[:, 0], point_cloud_np[:, 1], '.b', markersize=0.01)
    plt.pause(0.0001)

    fig.savefig("gif/"+str(idx)+"_vertex_idx.png")
    plt.close(fig)